In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision import models
import random
import copy
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/shomik/miniforge3/envs/pytorch_env/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/shomik/miniforge3/envs/pytorch_env/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can igno

In [2]:
num_random_seeds = 1
num_awp_models = 10

NNEURON = 100
NLAYER = 1
NEPOCH = 200
BETA1 = 0.9
BETA2 = 0.999
TRAINLR = 0.001
SEARCHLR = 0.001
SEARCHMOMENTUM = 0.9
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NUM_AWP = 1000

TARGET_VARIABLE = "qualified_gagne_3"
prediction_output = "predictions/obermeyer/awp_"+TARGET_VARIABLE+".csv"

In [3]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
#prediction_output = "predictions/obermeyer/bootstrap_"+MODEL_CLASS+"_"+TARGET_VARIABLE+".csv"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'gagne_sum_t', 'cost_t']

In [4]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[TARGET_VARIABLE]

In [5]:
## Multi-layer preceptron with weight perturbation
class MLP(nn.Module):
    def __init__(self, nn_arch):
        super(MLP, self).__init__()
        self.nfeature, self.nclass, self.nneuron, self.nlayer = nn_arch

        self.read_in = nn.Linear(self.nfeature, self.nneuron)
        self.ff = nn.Linear(self.nneuron, self.nneuron)
        self.read_out = nn.Linear(self.nneuron, self.nclass)

    def forward(self, x):
        x = self.read_in(x)
        for i in range(self.nlayer):
            x = F.relu(self.ff(x))

        logits = self.read_out(x)
        return logits

def train_model(model, X, y, epoch, optimizer, criterion, device):
    for i in range(epoch):
        model.train()
        optimizer.zero_grad()  # Setting our stored gradients equal to zero
        outputs = model(torch.Tensor(X).to(device))
        loss = criterion(torch.squeeze(outputs), torch.Tensor(y).type(torch.LongTensor).to(device))
        loss.backward()  # Computes the gradient of the given tensor w.r.t. graph leaves
        optimizer.step()

    return model

def evaluate(model, X, y, criterion, device):
    model.eval()
    logits = model(torch.Tensor(X).to(device))
    return criterion(torch.squeeze(logits), torch.Tensor(y).type(torch.LongTensor).to(device)).item()

def predict(model, X, class_target=1):
    model.eval()
    logits = model(torch.Tensor(X).to(device))
    pred = F.softmax(logits, dim=1)
    return pred[:, class_target].detach().numpy()

In [6]:
# Perturb the base model until the target instance has a prediction > likelihood tolerance, or the loss > loss tolerance
def get_awp_model(base_model, device, nn_arch, criterion, X_val, y_val, idx_target, class_target=1, pred_threshold=0.90, loss_tolerance=0.01):
    model = MLP(nn_arch).to(device)
    model.load_state_dict(copy.deepcopy(base_model.state_dict()))
    optimizer = torch.optim.SGD(model.parameters(), lr=SEARCHLR, momentum=SEARCHMOMENTUM)

    x_target = X_val[idx_target, :].reshape((1, X_val.shape[1]))
    target_logit = model(torch.Tensor(x_target).to(device))
    target_prob = torch.squeeze(F.softmax(target_logit, dim=1))
    baseline_loss = evaluate(model, X_val, y_val, criterion, device)
    val_loss = baseline_loss

    while target_prob[class_target]<pred_threshold and val_loss < baseline_loss + loss_tolerance:
        model.train()
        optimizer.zero_grad()
        target_logit = model(torch.Tensor(x_target).to(device))
        target_logit = torch.squeeze(target_logit)
        loss = -target_logit[class_target]
        loss.backward()
        optimizer.step()

        val_loss = evaluate(model, X_val, y_val, criterion, device)
        target_logit = model(torch.Tensor(x_target).to(device))
        target_prob = torch.squeeze(F.softmax(target_logit, dim=1))
    return model

In [7]:
output = []

for random_seed in range(num_random_seeds):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)
    
    cost = X_test["cost_t"].to_numpy()
    gagne = X_test["gagne_sum_t"].to_numpy()
    person_id = X_test['person_id'].to_numpy()
    
    X_train = X_train.drop(columns=other_variables).to_numpy()
    y_train = y_train.to_numpy()
    X_val = X_val.drop(columns=other_variables).to_numpy()
    y_val = y_val.to_numpy()
    X_test = X_test.drop(columns=other_variables).to_numpy()
    y_test = y_test.to_numpy()

    np.random.seed(random_seed)
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    nfeature = X_train.shape[1]
    nclass = len(set(y_train))
    nn_arch = [nfeature, nclass, NNEURON, NLAYER]

    base_model = MLP(nn_arch).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(base_model.parameters(), betas=[BETA1, BETA2], lr=TRAINLR)
    base_model = train_model(base_model, X_train, y_train, NEPOCH, optimizer, criterion, device)

    predictions = {}
    training_loss = {}
    validation_loss = {}

    for i in tqdm(range(num_awp_models)):
        model = get_awp_model(base_model, device, nn_arch, criterion, X_val, y_val, i)
        
        predictions[f'm_{i+1}'] = predict(model, X_test)
        training_loss[f'm_{i+1}'] = evaluate(model, X_train, y_train, criterion, device)
        validation_loss[f'm_{i+1}'] = evaluate(model, X_val, y_val, criterion, device)

    predictions_df = pd.concat([
        pd.DataFrame(predictions),
        pd.DataFrame(training_loss, index=[0]),
        pd.DataFrame(validation_loss, index=[0])]).reset_index(drop=True)
    
    predictions_df["y"] = np.concatenate([y_test, [np.nan, np.nan]])
    predictions_df["person_id"] = np.concatenate([person_id, [-2, -1]]) # -1 indicates validation loss, -2 indicates training loss
    predictions_df['cost_t'] = np.concatenate([cost, [np.nan, np.nan]]) 
    predictions_df['gagne_sum_t'] = np.concatenate([gagne, [np.nan, np.nan]])
    predictions_df["seed"] = random_seed

    output.append(predictions_df)
    
output = pd.concat(output)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 48.35it/s]


In [8]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,y,person_id,cost_t,gagne_sum_t,seed
0,0.065066,0.164047,0.110865,0.109792,0.112942,0.121622,0.065066,0.123239,0.122952,0.123007,0.0,2545,5300.0,1.0,0
1,0.194622,0.339797,0.444048,0.310816,0.318208,0.336618,0.194622,0.341167,0.340488,0.340558,0.0,8198,2700.0,2.0,0
2,0.040452,0.059029,0.125394,0.075393,0.056475,0.082874,0.040452,0.084641,0.084333,0.084402,0.0,46461,5200.0,0.0,0
3,0.006959,0.010036,0.013319,0.022842,0.009648,0.010428,0.006959,0.014886,0.014804,0.014838,0.0,30620,1300.0,0.0,0
4,0.237334,0.473250,0.360388,0.356583,0.369714,0.310749,0.237334,0.311333,0.310994,0.311054,0.0,47418,2200.0,2.0,0


In [9]:
output.to_csv(prediction_output, index=False)